In [1]:
# Give access to utils
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [22]:
import requests

search_include = [
    'cql.keywords any climate',
    'cql.keywords any earth'
]

# We apply this filter to be able to get the PDFs of each
search_filter = [
    'dc.publisher=Nature Publishing Group'
]

#'prism.productCode' to further narrow down on which nature

cql_query = '(' + ' OR '.join(search_include) + ') AND ' + ' AND '.join(search_filter)
print cql_query

data = {
    'httpAccept': 'application/json',
    'queryType': 'cql',
    'startRecord': '10970000',
    'query': cql_query
    
}
url = 'http://api.nature.com/content/opensearch/request'
response = requests.get(url, params=data)
body = response.json()

(cql.keywords any climate OR cql.keywords any earth) AND dc.publisher=Nature Publishing Group


In [23]:
#body['feed']['sru:numberOfRecords']
body['feed']['opensearch:startIndex']

KeyError: 'opensearch:startIndex'

In [24]:
# OPTIONAL

import pyperclip, webbrowser
# to do this little trick you need to pip install pyperclip

def browseJSON(json_str):
    pyperclip.copy(json_str)
    webbrowser.open_new_tab('https://jsonformatter.curiousconcept.com/')
    
browseJSON(response.text)

# then paste in your clipboard

In [ ]:
results = body['feed']['entry']

# An example result:
example = results[6]
url = example['link']

In [ ]:
# The URL of the example result is a redirect
import webbrowser
url
webbrowser.open_new_tab(url)

In [ ]:
# Get the redirected page - hopefully it should begin with nature.com
response = requests.get(url, params=data)
redirected_url = response.url
print redirected_url

In [ ]:
# Build the URL of the HTML page with the full article

doc_id = redirected_url.split('/')[-1]
article_url_fmt = 'http://www.nature.com.ezp-prod1.hul.harvard.edu/articles/%s'
article_url = article_url_fmt % doc_id
print article_url

In [ ]:
"""
OLD WAY: IGNORE

# Download the PDF file. If this doesn't work then the cookie is expired.
# In this case, copy the CURL from Chrome and use it to make a new .sh file
import subprocess
import os
cwd = os.getcwd()

env = {'PDF_URL': pdf_url, 'OUT_FILE': doc_id + '.pdf'}
with open('python-sh.log', 'a') as log2:
    p = subprocess.Popen('./nature-curl.sh', shell=True, cwd=cwd, env=env,
                    stderr=log2, stdout=log2)
p.wait()
cwd"""

In [ ]:
# Cleaner, more sharable way of downloading the file
import requests
import util

cookies = util.get_cookies('../credentials/gabe_nature.json')
article_fname = util.download_file(article_url, cookies)

In [ ]:

# Now we just have to convert the PDF to text with pdfminer
# This assumes you have pdfminer on your python path

# Abandoned for HTML

OUTPUT_DIR = './corpus/'

from pdf2txt import main as maketxt
maketxt([
    'pdf2txt',
    '-o', OUTPUT_DIR + pdf_fname.split('.')[0] + '.txt',
    pdf_fname
])